In [ ]:
#problem31: Transitions and Transversions
#transition: A <-> G / C <-> T
#transversion: A <-> C, A<->T, G<->C, G<->T
#transition transversion 변이 처리


data = {}
with open("rosalind_tran.txt") as handle:
    for line in handle:
        line = line.strip()
        if line.startswith(">"):
            id = line[1:]
            data[id] = ""
        else:
            data[id] += line

s1, s2 = data.values()

def trans(s1, s2):
    transisions = 0
    transversions = 0 
    transition_pairs = {('A', 'G'), ('G', 'A'), ('C', 'T'), ('T', 'C') }
    i = 0
    
    while i < len(s1):
        if s1[i] != s2[i]:
            if (s1[i], s2[i]) in transition_pairs:
                transisions += 1
            else:
                transversions += 1
        i += 1
    result = transisions/transversions
    return result

print(f"{trans(s1, s2):.11f}")



2.15000000000


In [ ]:
#problem32: Completing a Tree
# n개의 노드가 있으면 (n-1)개의 간선만 있어야 tree
#사이클이 없어야 한다
# 모든 노드는 연결되어 있어야 한다
#이미 연결된 건 다시 연결하면 안 된다
#머리아파서 못하겠다 너무 어렵다.. 질문이 잘 이해가 안간다..


In [ ]:
#problem33: Catalan Numbers and RNA Secondary Structures
#다시 한 번 볼것
#카탈란 수(catalan number): 조합론에서 등장하는 수열
#non-crossing 구조의 가짓수를 셈
#RNA 2차구조 같이 교차가 없는 걸 전제로 구조 모델링 예측, 해석 가능
#이진 트리의 수, 올바른 괄호 쌍의 수
#교차를 고려하는 경우는 RNA 3차구조(pseudoknot 등), NP문제
#근데 이거 시간 너어무 오래 걸려서 로잘린 5분안에 못품!!

with open("rosalind_cat.txt") as handle:
    for line in handle:
        line = line.strip()
        if not line.startswith(">"):
            read += line


 # 상보적인 염기가 있냐 없냐
def is_complementary(a, b):
    return (a == "A" and b == "U") or (a == "G" and b == "C") or \
            (a == "U" and b == "A") or (a == "C" and b == "G")


#카탈란의 핵심: read[0]을 기준으로 이것과 상보적인 read[i]를 찾기
#그러면 나오는 안쪽 read[1:i] 안과 read[i:]의 구분짓기
#perfect match는 짝수여야 됨, 훌수랑 매치하면 안됨
#재귀함수
def secondary_structure(read):
    if len(read) == 0:
        return 1
    
    pair = 0
    for i in range(1, len(read), 2):
        if is_complementary(read[0], read[i]):
            inside = secondary_structure(read[1:i])
            outside = secondary_structure(read[i+1:])
            pair += (inside * outside) % 1_000_000
    return pair % 1_000_000

print(secondary_structure(read))
                


KeyboardInterrupt: 

In [ ]:
#prooblem33: 시간 개선, lru_cache 사용
##다시 볼 것

from functools import lru_cache

read = ""
with open("rosalind_cat.txt") as handle:
    for line in handle:
        line = line.strip()
        if not line.startswith(">"):
            read += line

#상보적인 염기쌍인지 확인
def is_complementary(a, b):
    return (a == "A" and b == "U") or (a == "U" and b == "A") or \
           (a == "C" and b == "G") or (a == "G" and b == "C")

#이렇게 하면 함수 결과를 저장해서 중복 계산 안함
#재귀함수나 동적 계획법 문제에 씀
@lru_cache(maxsize=None) #캐시 크기 무재한
def secondary_structure(start, end):
    if start >= end:
        return 1 

#인덱스 기반으로 바꿈
    pairings = 0
    for k in range(start + 1, end + 1, 2):
        if is_complementary(read[start], read[k]):
            left = secondary_structure(start + 1, k - 1)
            right = secondary_structure(k + 1, end)
            pairings += (left * right) % 1_000_000
    return pairings % 1_000_000

# 결과 출력
print(secondary_structure(0, len(read) - 1)) #0based 인덱스니까 -1


326720


In [ ]:
#problem34: Error Correction in Reads
#hamming distance: 다른 글자가 몇 개인가
#s는 2번 등장(한 번은 역상보일수도 아닐수도 있음)
#틀린 시퀀스가 있는데 hamming distance가 1, 한번만 등장
#다시볼것

dataset = {}
with open("rosalind_corr.txt") as handle:
    for line in handle:
        line = line.strip()
        if line.startswith(">"):
            id = line[1:]
            dataset[id] = ""
        else:
            dataset[id] += line

#역상보 서열 함수
def reverse_complement(sequence):
    rev_seq = sequence[::-1]
    result = rev_seq.replace("T", "a").replace("A", "t").replace("G", "c").replace("C", "g")
    return result.upper()

#hamming distance 계산 함수
def hamming_distance(s1, s2):
    result = sum(c1 != c2 for c1, c2 in zip(s1, s2))
    return result

#find error read
def find_error_read(dataset: dict):
    reads = [x for x in dataset.values()]
    rev_reads = [reverse_complement(x) for x in reads]
    all_reads = reads + rev_reads
    error_reads = []
    correct_reads = []

    #s = s 이거나 s = rev_Seq
    for read in set(reads):
        if all_reads.count(read) >= 2:
            correct_reads.append(read)
        else:
            error_reads.append(read)

    corrections = []
    for error in error_reads:
        for correct in correct_reads:
            if hamming_distance(error, correct) == 1:
                corrections.append(f"{error}->{correct}")
                break
            elif hamming_distance(error, reverse_complement(correct)) == 1:
                corrections.append(f"{error}->{reverse_complement(correct)}")
                break
    return corrections        

print(' '.join(find_error_read(dataset)))


...


In [ ]:
#problem35: Counting Phylogenetic Ancestors
#관계식: I = n - 2
#잎이 n 개면 내부 노드는 n - 2개
#무근본 unrooted tree 라서 그럼
#무근본 이진 트리는 모든 내부 노드 차수가 3(세 갈래로 분기)
#잎은 실제 관찰된 종(leaf nodes)
#뿌리가 정해지지 않아서 방향 없이 가는 계통수
#내부 노드는 공통으로 분기한 조상 종

n = int(input("3 <= n <= 10000"))
print(n - 2)